In [1]:
import pandas as pd
import math

class Irt:
    """Classe creata per applicare algoritmi di Item Response Theory ad oggetti che rappresentano le prove d'esame,
       i quali hanno una riga per studente e le colonne rappresentano gli item su cui applicare le funzioni."""
    
    __minVote = 0
    __maxVote = 10
    
    def __init__(self, path):        
        """Il costruttore prende come parametro il path del file csv da analizzare."""        
        self.__df = pd.read_csv(path, index_col = 0) # rendo la prima colonna l'indice della tabella
        self.numItems = len(self.__df.columns) # salvo il numero di items    
        
    def __getVoteInput(self,msg):
        """Prendo voto in input assicurandomi che sia intero e compreso nell'intervallo"""
        while True:
            try:
                vote = int(input(msg))
                if vote not in range(self.__minVote,self.__maxVote + 1):
                    raise ValueError
                break
            except ValueError:
                print("Input non intero o non compreso nel range (", self.__minVote, "," , self.__maxVote, ")")
        return vote
       
    def __getDiff(self, n):
        """Metodo usato per ottenere la lista di difficoltà degli esercizi"""
        
        lsDiff = pd.DataFrame(columns = self.__df.columns , index = ["Difficulty"])
        print("Insersci la difficoltà per i seguenti item: \n")
        for columns in lsDiff:
            lsDiff.loc["Difficulty",columns] = self.__getVoteInput(columns + ": ")
        return lsDiff 

    def __getDiscrim(self, n):
        """Metodo usato per ottenere la lista di discriminanti degli esercizi"""        
        
        lsDiscrim = pd.DataFrame(columns = self.__df.columns , index = ["Discriminant"])
        print("Insersci i discriminanti per i seguenti item: \n")
        for columns in lsDiscrim:
            lsDiscrim.loc["Discriminant",columns] = self.__getVoteInput(columns + ": ")
        return lsDiscrim

    def __getGuess(self, n):
        """Metodo usato per ottenere la lista di probabilità di indovinare esercizi"""    
        
        lsGuess = pd.DataFrame(columns = self.__df.columns , index = ["Guess"])
        print("Insersci le probabilità di indovinare i seguenti item: \n")
        for columns in lsGuess:
            lsGuess.loc["Guess",columns] = float(input(columns + ": "))
        return lsGuess  
    
    def __estimateAbility(self, lsDiff):
        """Metodo usato per fornire una stima delle abilità degli studenti tramite una media ponderata"""   
        
        lsAb = pd.DataFrame(index = self.__df.index , columns = ["Ability"])
        for index, row in self.__df.iterrows():
            ability = (row*(lsDiff.loc["Difficulty"])).sum()/lsDiff.loc["Difficulty"].sum()
            lsAb.loc[index,"Ability"] = round(ability, 2)
        return lsAb      
    
    def pl1(self):
        """Metodo usato per applicare il modello PL1 dell'Item Response Theory, esso usa solo il parametro delle difficoltà""" 
        
        lsDiff = self.__getDiff(self.numItems)
        lsAb   = self.__estimateAbility(lsDiff)
        dfProb = pd.DataFrame(columns=self.__df.columns, index=self.__df.index)
        for index, row in dfProb.iterrows():
            for columns in dfProb:
                ability    = lsAb.loc  [index,"Ability"]
                difficulty = lsDiff.loc["Difficulty",columns]
                res = (math.exp(ability - difficulty)) / (1 + math.exp(ability - difficulty))
                dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
        return dfProb

    def pl2(self):
        """Metodo usato per applicare il modello PL1 dell'Item Response Theory, esso usa le difficoltà ed i discriminanti"""  
        
        lsDiff    = self.__getDiff(self.numItems)
        lsAb      = self.__estimateAbility(lsDiff)
        print()
        lsDiscrim = self.__getDiscrim(self.numItems)
        dfProb = pd.DataFrame(columns=self.__df.columns, index=self.__df.index)
        for index, row in dfProb.iterrows():
            for columns in dfProb:
                ability      = lsAb.loc     [index,"Ability"]
                difficulty   = lsDiff.loc   ["Difficulty",columns]      
                discriminant = lsDiscrim.loc["Discriminant",columns]
                res = (math.exp(discriminant * (ability - difficulty))) / (1 + math.exp(discriminant * (ability - difficulty)))
                dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
        return dfProb

    def pl3(self):
        """Metodo usato per applicare il modello PL1 dell'Item Response Theory, esso usa le difficoltà, discriminanti e 
        le probabilità di indovinare gli esercizi"""        
        
        lsDiff    = self.__getDiff(self.numItems)
        lsAb      = self.__estimateAbility(lsDiff)
        print()
        lsDiscrim = self.__getDiscrim(self.numItems)
        print()
        lsGuess   = self.__getGuess(self.numItems)
        dfProb    = pd.DataFrame(columns=self.__df.columns, index=self.__df.index)
        
        for index, row in dfProb.iterrows():
            for columns in dfProb:
                ability      = lsAb.loc     [index,"Ability"]
                difficulty   = lsDiff.loc   ["Difficulty",columns]      
                discriminant = lsDiscrim.loc["Discriminant",columns]            
                guessProb    = lsGuess.loc  ["Guess",columns]
                res = guessProb + (1 - guessProb) * (math.exp(discriminant * (ability - difficulty))) / (1 + math.exp(discriminant * (ability - difficulty)))
                dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
        return dfProb    
    
    def getDf(self):
        """Restituisce il la tabella relativa al file csv utilizzato"""
        return self.__df    

In [2]:
test = Irt("testcsv.csv")
test.pl2()

Insersci la difficoltà per i seguenti item: 

Esercizio1: 5
Esercizio2: 6
Esercizio3: 7

Insersci i discriminanti per i seguenti item: 

Esercizio1: 1
Esercizio2: 1
Esercizio3: 1


,Esercizio1,Esercizio2,Esercizio3
Studente,,,
Marco,81.76%,62.25%,37.75%
Simone,54.24%,30.36%,13.82%
Davide,86.18%,69.64%,45.76%
